In [ ]:
1

1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json
import os
import numpy as np

In [ ]:
pd.set_option('display.max_columns', 20)

In [ ]:
# file_name = "/content/drive/MyDrive/DropBox Hackathon/LoL_Data/tournaments.json"
file_name = "/content/drive/MyDrive/LoL_Data/tournaments.json"

with open(file_name) as user_file:
    file = json.load(user_file)

In [ ]:
data = file.copy()

In [ ]:
# Initialize an empty list to store the extracted data
table_data = []

# Iterate through the JSON data and extract the required information
for league in data:
    league_id = league["leagueId"]
    trn_name = league["name"]
    trn_id = league["id"]

    for stage in league["stages"]:
        stage_name = stage["name"]

        for section in stage["sections"]:
            section_name = section["name"]

            for match in section["matches"]:
                match_id = match["id"]
                match_state = match["state"]



                for game in match["games"]:

                    # print(game['state'])
                    game_id = game["id"]

                    if game['state'] == 'completed':

                        for team in game["teams"]:

                            team_id = team["id"]
                            team_result = team["result"]["outcome"]

                            # print(team_id, team_result)
                            table_data.append([league_id, trn_name, trn_id, stage_name, section_name, match_id, game_id, team_id, team_result])

                        # break

                    # Append the extracted data to the list



In [ ]:
len(table_data)

49256

In [ ]:
table_data[:3]

[['109518549825754242',
  '#2 Summer 2023',
  '110733838935136200',
  'Groups',
  'Group A',
  '110733838936381392',
  '110733838936446929',
  '109981650516317055',
  'win'],
 ['109518549825754242',
  '#2 Summer 2023',
  '110733838935136200',
  'Groups',
  'Group A',
  '110733838936381392',
  '110733838936446929',
  '110733889090164854',
  'loss'],
 ['109518549825754242',
  '#2 Summer 2023',
  '110733838935136200',
  'Groups',
  'Group A',
  '110733838936381392',
  '110733838936446930',
  '110733889090164854',
  'loss']]

In [ ]:
# Create a DataFrame from the extracted data
table_df = pd.DataFrame(table_data, columns=["League ID", "Trn Name", "Trn ID", "Stage", "Section", "Match ID", "Game ID", "Team ID", "Outcome"])

# Display the final table
table_df.head()

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,win
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,loss
2,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,110733889090164854,loss
3,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,109981650516317055,win
4,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936446932,110733838936446933,110733877357854829,win


In [ ]:
table_df.shape

(49256, 9)

In [ ]:
# pd.set_option("display.max_rows", 20)

In [ ]:
# check if Stage and Section is the same
table_df.loc[table_df['Stage'] != table_df["Section"]].shape[0]

12112

## Get mapping table

In [ ]:
# file_name = "/content/drive/MyDrive/DropBox Hackathon/LoL_Data/mapping_data.json"
file_name = "/content/drive/MyDrive/LoL_Data/mapping_data.json"

with open(file_name) as user_file:
    mapping_data = json.load(user_file)

In [ ]:
mapping_data = pd.DataFrame(mapping_data)
mapping_data.head(5)

,esportsGameId,platformGameId,teamMapping,participantMapping
0,110378429158160389,ESPORTSTMNT01:3416295,"{'200': '105550026570060790', '100': '10555003...","{'3': '107492116585043595', '5': '105388980252..."
1,110471139171602607,ESPORTSTMNT02:3228341,"{'200': '105550005698683818', '100': '10948533...","{'3': '107560281476330464', '5': '107559633217..."
2,110535609417029691,ESPORTSTMNT04:2693199,"{'200': '108352305932141947', '100': '10969609...","{'3': '105536902415377888', '5': '109696579630..."
3,110847390580148354,ESPORTSTMNT01:3413275,"{'200': '99566406332987990', '100': '107700204...","{'3': '98767991808793901', '5': '1096429488383..."
4,110413246204026226,ESPORTSTMNT01:3412587,"{'200': '105550001032913831', '100': '10948533...","{'3': '107560281476330464', '5': '107559633217..."


In [ ]:
mapping_data.shape

(25283, 4)

In [ ]:
teams_id = pd.json_normalize(mapping_data['teamMapping']) ## much faster
teams_id.head()

,200,100
0,105550026570060790,105550033967461806
1,105550005698683818,109485335453835911
2,108352305932141947,109696092218588987
3,99566406332987990,107700204561086446
4,105550001032913831,109485335453835911


In [ ]:
mapping_data = pd.concat([mapping_data.drop(['teamMapping', 'participantMapping'], axis=1), teams_id], axis=1)
mapping_data.head()

,esportsGameId,platformGameId,200,100
0,110378429158160389,ESPORTSTMNT01:3416295,105550026570060790,105550033967461806
1,110471139171602607,ESPORTSTMNT02:3228341,105550005698683818,109485335453835911
2,110535609417029691,ESPORTSTMNT04:2693199,108352305932141947,109696092218588987
3,110847390580148354,ESPORTSTMNT01:3413275,99566406332987990,107700204561086446
4,110413246204026226,ESPORTSTMNT01:3412587,105550001032913831,109485335453835911


In [ ]:
mapping_data.shape

(25283, 4)

In [ ]:
table_df.head(2)

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,win
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,loss


In [ ]:
master_table_df = pd.merge(table_df, mapping_data, left_on = 'Game ID', right_on = 'esportsGameId', how = 'left')
master_table_df

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome,esportsGameId,platformGameId,200,100
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,win,110733838936446929,ESPORTSTMNT03:3196037,110733889090164854,109981650516317055
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,loss,110733838936446929,ESPORTSTMNT03:3196037,110733889090164854,109981650516317055
2,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,110733889090164854,loss,110733838936446930,ESPORTSTMNT03:3196049,109981650516317055,110733889090164854
3,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,109981650516317055,win,110733838936446930,ESPORTSTMNT03:3196049,109981650516317055,110733889090164854
4,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936446932,110733838936446933,110733877357854829,win,110733838936446933,ESPORTSTMNT03:3196058,110534732558665825,110733877357854829
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49259,101382741235120470,lla-closing-2020,104237136838348356,Playoffs,Playoffs,104237136865807906,104237136865807909,98767991935149427,win,NaN,NaN,NaN,NaN
49260,101382741235120470,lla-closing-2020,104237136838348356,Playoffs,Playoffs,104237136865807906,104237136865807910,98767991935149427,win,NaN,NaN,NaN,NaN
49261,101382741235120470,lla-closing-2020,104237136838348356,Playoffs,Playoffs,104237136865807906,104237136865807910,101383792559569368,loss,NaN,NaN,NaN,NaN
49262,101382741235120470,lla-closing-2020,104237136838348356,Playoffs,Playoffs,104237136865807906,104237136865807911,101383792559569368,loss,NaN,NaN,NaN,NaN


In [ ]:
master_table_df.loc[master_table_df['esportsGameId'].isnull()].shape
# mapping of esportsGameID between tournaments.json (table_df) and mapping_data.json
# wasn't possible for 1844/2 ~ 900 games

(1844, 13)

In [ ]:
master_table_df = master_table_df.loc[master_table_df['esportsGameId'].notnull()]
master_table_df.shape

(47420, 13)

In [ ]:
master_table_df.head()

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome,esportsGameId,platformGameId,200,100
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,win,110733838936446929,ESPORTSTMNT03:3196037,110733889090164854,109981650516317055
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,loss,110733838936446929,ESPORTSTMNT03:3196037,110733889090164854,109981650516317055
2,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,110733889090164854,loss,110733838936446930,ESPORTSTMNT03:3196049,109981650516317055,110733889090164854
3,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,109981650516317055,win,110733838936446930,ESPORTSTMNT03:3196049,109981650516317055,110733889090164854
4,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936446932,110733838936446933,110733877357854829,win,110733838936446933,ESPORTSTMNT03:3196058,110534732558665825,110733877357854829


## Make a table from the game data files

In [ ]:
mylist = []
# dir_path = '/content/drive/MyDrive/DropBox Hackathon/LoL_Data_games_small/'
dir_path = '/content/drive/MyDrive/LoL_Data_games_small/'

for idx, filename in enumerate(os.listdir(dir_path)):

    with open(os.path.join(dir_path, filename), 'r') as file:
        mylist.append(json.load(file))

    if idx % 100 == 0: print(idx)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
len(mylist)

23706

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_rows', 20)

In [ ]:
games_df = pd.DataFrame(mylist)
games_df.head(3)

,eventTime,level_100,level_200,xp_100,xp_200,totalGold_100,totalGold_200,attackSpeed_100,attackSpeed_200,respawnTimer_100,...,deaths_100,inhibKills_200,towerKills_200,baronKills_200,dragonKills_200,assists_200,championsKills_200,deaths_200,winning_team,platformGameId
0,2020-09-26T13:02:25.566Z,"[16, 16, 17, 15, 13]","[17, 17, 15, 16, 12]","[15720, 15203, 17901, 14571, 10995]","[17462, 16729, 14094, 15442, 9229]",61717.0,55328.0,"[164, 186, 165, 127, 142]","[141, 161, 133, 160, 120]",0,...,13.0,0.0,2.0,0.0,2.0,33.0,13.0,13.0,100.0,ESPORTSTMNT03:1531088
1,2020-09-25T09:56:09.021Z,"[18, 17, 17, 15, 14]","[16, 15, 16, 14, 13]","[18847, 16710, 16520, 13559, 11658]","[16105, 13406, 14732, 12067, 9974]",63492.0,54175.0,"[182, 161, 137, 193, 124]","[174, 173, 146, 258, 141]",0,...,8.0,0.0,2.0,0.0,1.0,12.0,8.0,27.0,100.0,ESPORTSTMNT03:1530963
2,2020-09-26T10:52:31.724Z,"[14, 14, 13, 14, 13]","[16, 15, 17, 15, 13]","[11752, 12728, 11350, 11976, 10005]","[16262, 13372, 17574, 14324, 10853]",47929.0,60218.0,"[125, 125, 116, 272, 129]","[176, 162, 158, 146, 120]",0,...,30.0,2.0,11.0,1.0,4.0,46.0,30.0,10.0,200.0,ESPORTSTMNT03:1531064


In [ ]:
games_df.shape

(23706, 33)

In [ ]:
games_df.to_excel('/content/drive/MyDrive/2_data_files/games_df.xlsx', index = False)

## Crazy full merge

In [ ]:
master_table_df_games = pd.merge(master_table_df, games_df, on = 'platformGameId',  how = 'left')
master_table_df_games

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome,esportsGameId,...,championsKills_100,deaths_100,inhibKills_200,towerKills_200,baronKills_200,dragonKills_200,assists_200,championsKills_200,deaths_200,winning_team
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,win,110733838936446929,...,22.0,7.0,0.0,1.0,0.0,0.0,8.0,7.0,22.0,100.0
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,loss,110733838936446929,...,22.0,7.0,0.0,1.0,0.0,0.0,8.0,7.0,22.0,100.0
2,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,110733889090164854,loss,110733838936446930,...,14.0,16.0,1.0,9.0,1.0,3.0,33.0,16.0,14.0,200.0
3,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,109981650516317055,win,110733838936446930,...,14.0,16.0,1.0,9.0,1.0,3.0,33.0,16.0,14.0,200.0
4,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936446932,110733838936446933,110733877357854829,win,110733838936446933,...,24.0,6.0,0.0,1.0,0.0,1.0,17.0,6.0,24.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47415,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150689,105548000936710641,loss,105814193355150689,...,29.0,33.0,1.0,6.0,0.0,4.0,90.0,32.0,29.0,200.0
47416,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150690,105548000936710641,loss,105814193355150690,...,31.0,14.0,0.0,4.0,0.0,0.0,31.0,14.0,31.0,100.0
47417,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150690,105543843212923183,win,105814193355150690,...,31.0,14.0,0.0,4.0,0.0,0.0,31.0,14.0,31.0,100.0
47418,101382741235120470,lla-closing-2020,104237136838348356,Playoffs,Playoffs,104237136865807900,104237136865807904,98767991935149427,win,104237136865807904,...,13.0,7.0,0.0,1.0,0.0,2.0,17.0,7.0,13.0,100.0


In [ ]:
master_table_df_games.loc[master_table_df_games["assists_200"].notnull()].shape
# should be 10k rows <-- 5k games from games_df
# should be 15500 rows <-- 7750 games from games_df
# should be 30000 rows <-- 15000 games from games_df
# # should be 40000 rows <-- 20000 games from games_df

(47410, 45)

In [ ]:
# get data in good form from here

# df = master_table_df_games.loc[master_table_df_games["assists_200"].notnull()].head(500)
df = master_table_df_games.loc[master_table_df_games["assists_200"].notnull()]
df

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome,esportsGameId,...,championsKills_100,deaths_100,inhibKills_200,towerKills_200,baronKills_200,dragonKills_200,assists_200,championsKills_200,deaths_200,winning_team
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,win,110733838936446929,...,22.0,7.0,0.0,1.0,0.0,0.0,8.0,7.0,22.0,100.0
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,loss,110733838936446929,...,22.0,7.0,0.0,1.0,0.0,0.0,8.0,7.0,22.0,100.0
2,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,110733889090164854,loss,110733838936446930,...,14.0,16.0,1.0,9.0,1.0,3.0,33.0,16.0,14.0,200.0
3,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,109981650516317055,win,110733838936446930,...,14.0,16.0,1.0,9.0,1.0,3.0,33.0,16.0,14.0,200.0
4,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936446932,110733838936446933,110733877357854829,win,110733838936446933,...,24.0,6.0,0.0,1.0,0.0,1.0,17.0,6.0,24.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47415,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150689,105548000936710641,loss,105814193355150689,...,29.0,33.0,1.0,6.0,0.0,4.0,90.0,32.0,29.0,200.0
47416,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150690,105548000936710641,loss,105814193355150690,...,31.0,14.0,0.0,4.0,0.0,0.0,31.0,14.0,31.0,100.0
47417,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150690,105543843212923183,win,105814193355150690,...,31.0,14.0,0.0,4.0,0.0,0.0,31.0,14.0,31.0,100.0
47418,101382741235120470,lla-closing-2020,104237136838348356,Playoffs,Playoffs,104237136865807900,104237136865807904,98767991935149427,win,104237136865807904,...,13.0,7.0,0.0,1.0,0.0,2.0,17.0,7.0,13.0,100.0


In [ ]:
df.columns

Index(['League ID', 'Trn Name', 'Trn ID', 'Stage', 'Section', 'Match ID',
       'Game ID', 'Team ID', 'Outcome', 'esportsGameId', 'platformGameId',
       '200', '100', 'eventTime', 'level_100', 'level_200', 'xp_100', 'xp_200',
       'totalGold_100', 'totalGold_200', 'attackSpeed_100', 'attackSpeed_200',
       'respawnTimer_100', 'respawnTimer_200', 'armor_100', 'armor_200',
       'magicResist_100', 'magicResist_200', 'armorPenetration_100',
       'armorPenetration_200', 'inhibKills_100', 'towerKills_100',
       'baronKills_100', 'dragonKills_100', 'assists_100',
       'championsKills_100', 'deaths_100', 'inhibKills_200', 'towerKills_200',
       'baronKills_200', 'dragonKills_200', 'assists_200',
       'championsKills_200', 'deaths_200', 'winning_team'],
      dtype='object')

In [ ]:
df[['League ID', 'Trn Name', 'Trn ID', 'Stage', 'Section', 'Match ID',
       'Game ID', 'Team ID', 'Outcome']].to_excel("/content/drive/MyDrive/2_data_files/trn_names.xlsx", index = False)

## Get date information

In [ ]:
df["eventTime_split"] = df["eventTime"].str.split("-")
df["date"] = df["eventTime"].str[:11]

df.insert(loc = 10, column = 'year', value =  df["eventTime_split"].str[0].astype(int))
df.insert(loc = 10, column = 'month', value =  df["eventTime_split"].str[1].astype(int))
df.insert(loc = 10, column = 'day', value =  df["eventTime_split"].str[2].str.split("T").str[0].astype(int))

df.drop(['eventTime_split'], axis = 1, inplace = True)

<ipython-input-36-0987f7867371>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["eventTime_split"] = df["eventTime"].str.split("-")
<ipython-input-36-0987f7867371>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = df["eventTime"].str[:11]
<ipython-input-36-0987f7867371>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

## Change outcome to 0 and 1

In [ ]:
# pd.set_option('display.max_rows', 20)

In [ ]:
df["Outcome"].value_counts()

win        23705
loss       23703
forfeit        2
Name: Outcome, dtype: int64

In [ ]:
df["Outcome"] = df["Outcome"].map({"win" : 1, "loss" : 0, "forfeit" : 0})

<ipython-input-39-68f3cb81329c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outcome"] = df["Outcome"].map({"win" : 1, "loss" : 0, "forfeit" : 0})


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47410 entries, 0 to 47419
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   League ID             47410 non-null  object 
 1   Trn Name              47410 non-null  object 
 2   Trn ID                47410 non-null  object 
 3   Stage                 47410 non-null  object 
 4   Section               47410 non-null  object 
 5   Match ID              47410 non-null  object 
 6   Game ID               47410 non-null  object 
 7   Team ID               47410 non-null  object 
 8   Outcome               47410 non-null  int64  
 9   esportsGameId         47410 non-null  object 
 10  day                   47410 non-null  int64  
 11  month                 47410 non-null  int64  
 12  year                  47410 non-null  int64  
 13  platformGameId        47410 non-null  object 
 14  200                   47372 non-null  object 
 15  100                

### make df readable

In [ ]:
for col in ['level', 'xp', 'totalGold', 'attackSpeed', 'respawnTimer', 'armor', 'magicResist',
            'armorPenetration', 'inhibKills', 'towerKills', 'baronKills', 'dragonKills',
            'assists', 'championsKills', 'deaths']:

            df[f'{col}_own'] = np.where( df["Team ID"] == df['200'], df[f"{col}_200"], df[f"{col}_100"] )
            df[f'{col}_opp'] = np.where( df["Team ID"] == df['100'], df[f"{col}_200"], df[f"{col}_100"] )

<ipython-input-41-3d99d71f307b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col}_own'] = np.where( df["Team ID"] == df['200'], df[f"{col}_200"], df[f"{col}_100"] )
<ipython-input-41-3d99d71f307b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col}_opp'] = np.where( df["Team ID"] == df['100'], df[f"{col}_200"], df[f"{col}_100"] )
<ipython-input-41-3d99d71f307b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [ ]:
# drop redundant columns
df = df.loc[:,~df.columns.str.endswith('100')]
df = df.loc[:,~df.columns.str.endswith('200')]
df

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome,esportsGameId,...,baronKills_own,baronKills_opp,dragonKills_own,dragonKills_opp,assists_own,assists_opp,championsKills_own,championsKills_opp,deaths_own,deaths_opp
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,1,110733838936446929,...,1.0,0.0,2.0,0.0,50.0,8.0,22.0,7.0,7.0,22.0
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,0,110733838936446929,...,0.0,1.0,0.0,2.0,8.0,50.0,7.0,22.0,22.0,7.0
2,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,110733889090164854,0,110733838936446930,...,0.0,1.0,0.0,3.0,26.0,33.0,14.0,16.0,16.0,14.0
3,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446930,109981650516317055,1,110733838936446930,...,1.0,0.0,3.0,0.0,33.0,26.0,16.0,14.0,14.0,16.0
4,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936446932,110733838936446933,110733877357854829,1,110733838936446933,...,1.0,0.0,2.0,1.0,61.0,17.0,24.0,6.0,6.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47415,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150689,105548000936710641,0,105814193355150689,...,0.0,2.0,4.0,2.0,90.0,77.0,32.0,29.0,29.0,33.0
47416,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150690,105548000936710641,0,105814193355150690,...,1.0,0.0,4.0,0.0,63.0,31.0,31.0,14.0,14.0,31.0
47417,105266118689416013,baltic_masters_2021_spring,105562379823435874,Playoffs,Playoffs,105814193355150685,105814193355150690,105543843212923183,1,105814193355150690,...,0.0,1.0,0.0,4.0,31.0,63.0,14.0,31.0,31.0,14.0
47418,101382741235120470,lla-closing-2020,104237136838348356,Playoffs,Playoffs,104237136865807900,104237136865807904,98767991935149427,1,104237136865807904,...,2.0,0.0,3.0,2.0,26.0,17.0,13.0,7.0,7.0,13.0


## Get averages from the lists

In [ ]:
def get_avg_val(row, col_name, team_type):
    return np.mean(row[f'{col_name}_{team_type}']).round(3)

for col_name in ["xp", "level", "attackSpeed", "respawnTimer",
                 "armor", "magicResist", "armorPenetration"]:

                for team_type in ['own', 'opp']:

                    df[f'{col_name}_{team_type}_avg'] = df.apply(get_avg_val, args = [col_name, team_type], axis = 1)

                    # drop the columns which had lists
                    df.drop([f'{col_name}_{team_type}'], axis=1, inplace = True)

In [ ]:
# df

In [ ]:
df.describe()

,Outcome,day,month,year,winning_team,totalGold_own,totalGold_opp,inhibKills_own,inhibKills_opp,towerKills_own,...,attackSpeed_own_avg,attackSpeed_opp_avg,respawnTimer_own_avg,respawnTimer_opp_avg,armor_own_avg,armor_opp_avg,magicResist_own_avg,magicResist_opp_avg,armorPenetration_own_avg,armorPenetration_opp_avg
count,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,...,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.000000,47410.0,47410.0
mean,0.500000,15.564902,4.870660,2021.870618,147.255853,57163.984012,57165.871567,0.905104,0.905358,5.789158,...,169.449930,169.449357,7.608254,7.607150,128.348644,128.351504,76.983139,76.982109,0.0,0.0
std,0.500005,8.751159,2.494935,0.989604,49.925166,11321.730155,11321.157150,1.067943,1.067965,3.555458,...,18.673219,18.671454,9.073970,9.073551,29.069810,29.068955,20.268313,20.266852,0.0,0.0
min,0.000000,1.000000,1.000000,2020.000000,100.000000,23183.000000,23183.000000,0.000000,0.000000,0.000000,...,116.600000,116.600000,0.000000,0.000000,59.000000,59.000000,35.400000,35.400000,0.0,0.0
25%,0.000000,8.000000,3.000000,2021.000000,100.000000,49431.500000,49434.000000,0.000000,0.000000,2.000000,...,156.200000,156.200000,0.000000,0.000000,107.600000,107.600000,62.000000,62.000000,0.0,0.0
50%,0.500000,16.000000,6.000000,2022.000000,100.000000,56822.000000,56826.000000,1.000000,1.000000,6.000000,...,167.800000,167.800000,4.609000,4.607000,124.800000,124.800000,73.800000,73.800000,0.0,0.0
75%,1.000000,23.000000,7.000000,2023.000000,200.000000,64302.000000,64306.750000,2.000000,2.000000,9.000000,...,181.000000,181.000000,12.578250,12.575750,145.200000,145.200000,88.600000,88.600000,0.0,0.0
max,1.000000,31.000000,12.000000,2023.000000,200.000000,120870.000000,120870.000000,10.000000,10.000000,17.000000,...,276.000000,276.000000,58.777000,58.777000,347.800000,347.800000,196.200000,196.200000,0.0,0.0


In [ ]:
# df.to_excel("final_data_sample.xlsx")

In [ ]:
df.columns

Index(['League ID', 'Trn Name', 'Trn ID', 'Stage', 'Section', 'Match ID',
       'Game ID', 'Team ID', 'Outcome', 'esportsGameId', 'day', 'month',
       'year', 'platformGameId', 'eventTime', 'winning_team', 'date',
       'totalGold_own', 'totalGold_opp', 'inhibKills_own', 'inhibKills_opp',
       'towerKills_own', 'towerKills_opp', 'baronKills_own', 'baronKills_opp',
       'dragonKills_own', 'dragonKills_opp', 'assists_own', 'assists_opp',
       'championsKills_own', 'championsKills_opp', 'deaths_own', 'deaths_opp',
       'xp_own_avg', 'xp_opp_avg', 'level_own_avg', 'level_opp_avg',
       'attackSpeed_own_avg', 'attackSpeed_opp_avg', 'respawnTimer_own_avg',
       'respawnTimer_opp_avg', 'armor_own_avg', 'armor_opp_avg',
       'magicResist_own_avg', 'magicResist_opp_avg',
       'armorPenetration_own_avg', 'armorPenetration_opp_avg'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47410 entries, 0 to 47419
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   League ID                 47410 non-null  object 
 1   Trn Name                  47410 non-null  object 
 2   Trn ID                    47410 non-null  object 
 3   Stage                     47410 non-null  object 
 4   Section                   47410 non-null  object 
 5   Match ID                  47410 non-null  object 
 6   Game ID                   47410 non-null  object 
 7   Team ID                   47410 non-null  object 
 8   Outcome                   47410 non-null  int64  
 9   esportsGameId             47410 non-null  object 
 10  day                       47410 non-null  int64  
 11  month                     47410 non-null  int64  
 12  year                      47410 non-null  int64  
 13  platformGameId            47410 non-null  object 
 14  eventT

In [ ]:
df.head(2)

,League ID,Trn Name,Trn ID,Stage,Section,Match ID,Game ID,Team ID,Outcome,esportsGameId,...,attackSpeed_own_avg,attackSpeed_opp_avg,respawnTimer_own_avg,respawnTimer_opp_avg,armor_own_avg,armor_opp_avg,magicResist_own_avg,magicResist_opp_avg,armorPenetration_own_avg,armorPenetration_opp_avg
0,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,109981650516317055,1,110733838936446929,...,185.2,155.2,0.000,15.025,142.6,101.8,76.4,55.8,0.0,0.0
1,109518549825754242,#2 Summer 2023,110733838935136200,Groups,Group A,110733838936381392,110733838936446929,110733889090164854,0,110733838936446929,...,155.2,185.2,15.025,0.000,101.8,142.6,55.8,76.4,0.0,0.0


In [ ]:
df['League ID'] = df['League ID'].astype(int)
df['Trn ID'] = df['Trn ID'].astype(int)
df['Match ID'] = df['Match ID'].astype(int)
df['Game ID'] = df['Game ID'].astype(int)
df['Team ID'] = df['Team ID'].astype(int)
# df['event_time'] = df['event_time'].astype(datetime64)
# object

In [ ]:
# df.info()

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in.
    for col in props.columns:
        if (props[col].dtype != object) :  # Exclude strings

            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)

            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()

            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all():
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)

            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True


            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)

            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)

            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")

    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
df, NAlist = reduce_mem_usage(df)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 17.362060546875  MB
******************************
Column:  League ID
dtype before:  int64
dtype after:  uint64
******************************
******************************
Column:  Trn ID
dtype before:  int64
dtype after:  uint64
******************************
******************************
Column:  Match ID
dtype before:  int64
dtype after:  uint64
******************************
******************************
Column:  Game ID
dtype before:  int64
dtype after:  uint64
******************************
******************************
Column:  Team ID
dtype before:  int64
dtype after:  uint64
******************************
******************************
Column:  Outcome
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  day
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  month
dtype before:  int64
dtype after:  uint8
********

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head(3)

In [ ]:
pd.set_option('display.max_columns', 20)

In [ ]:
df.to_excel("/content/drive/MyDrive/final_data.xlsx", index = False)

KeyboardInterrupt: ignored